<a href="https://colab.research.google.com/github/LakshaySharma10/S4E03-Steel-plate-fault-detection/blob/main/Steel_Plate_Fault.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'playground-series-s4e3:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F68699%2F7659021%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240326%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240326T042007Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2ce0f299f8b881ed460604162ebec6c1ae981ab26adc3a5a74a91153b2bffd803e89ef625de911723d3f4f6ab0f025b2695b53e0f4634e6622be6c91c228e08f3f14792dbaa976fc4d6585be145dbc1d52d60684c14397d0e7cb7f0bf8b541c27e067a25619b031876632a9c130ca6f41945bb8615b81910269e94c8ff8487bbf4410e0bef438080d9e1e2f17318a7d55c92e375213570e70fe095b466e4e125d562903230f5c8cadbfdb666c47d30b0c9586022750c5afebcf2684570afe55ff82613cf59f27fafbecbeefd9bac9a3ba66060be8d99d10d381942abd700890ae521fb87f554e07aff39851c563dd8a917de355ac692999b93c31f1df027dacd,faulty-steel-plates:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2363%2F3972%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240326%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240326T042007Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D58eef0077798a9d986bd58bd2ca7f2c801e8d0ce7a7ab8b9f4f8229c80d9d8e42fb3fdce283dfbeb4685027d9747b8bd601b6da7f9068b15a9320bb9de97b2a0b09d8d8d6d0e20572a18d0c24901c2c883d6e8308cfb352992e38001f6e2d8de0f3787065ba27c46015407fb0f69f8ba433547272bdbc5f37f22e0961ef532bf8c31d169f4e9edc04b2bad432fadebfb5ecf5a39fa8340595b7d9d8b88138a0ed93b28d34c56be5df45904adff5d58cd981672ab05247e0b30ab900b518cfa6773758a520fd10711f10a1d9dc3180fcb1e2962d2b0eec68e7dd8a772bf0bec84c8dab439a01cc036e169a228d31f0b294cd89c009ffd7d622e4f45b390e1c679,steel-plate-fault-submission:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4559448%2F7789584%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240326%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240326T042007Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D56565bff1565d0fa630fd74dbf0225f21473ba221461cbdd1bb6b770e70acac818813e225831795b581b3789f0a4816d1c43fd775c0629336c72da6c550de7a0b2957b836a4fa0318ddd2fdff673499f4b55fb40d7fb5fff56b8647346d79fb9b69dba11fe95a0e21772e1276202bf8b20df86866923244ce39fdcec62b485b71a556fd01a797e7b01cefc2459d466e7dcb3093a374f43cb50e810619ead9df84f6f545a53f8f775c06b3d15dcb797e1c6a3319bef77c87fd811b72190874a681f738c2084c90903767ec36dc127d3c0bd51ad3aa3a207937c1c42ebe26d3227ff8fcf9d804952152e3b6b05ae0e3a723f02751d0271d197eed5c66f88993f99'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


# Load Libraries

In [ ]:
!pip install iterative-stratification cmaes -q

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

import sklearn
import warnings

warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import tensorflow as tf
import random as rn
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

import os

os.listdir("/kaggle/input")
os.environ["PYTHONHASHSEED"] = "325"
rn.seed(279)
tf.random.set_seed(23)

# Load Data

In [ ]:
df_train = pd.read_csv('/kaggle/input/playground-series-s4e3/train.csv')
df_org = pd.read_csv('/kaggle/input/faulty-steel-plates/faults.csv')
df_test = pd.read_csv('/kaggle/input/playground-series-s4e3/test.csv')
df_sub  = pd.read_csv('/kaggle/input/playground-series-s4e3/sample_submission.csv')

In [ ]:
df_train=df_train.drop(['id'],axis=1)
df_test_ids = df_test['id']
df_test=df_test.drop(['id'],axis=1)
df_combined = pd.concat([df_train, df_org], ignore_index=True)

In [ ]:
label_columns=['Pastry','Z_Scratch', 'K_Scatch',   'Stains',   'Dirtiness','Bumps','Other_Faults']

In [ ]:
df_train=df_combined
y_train=df_train[label_columns]
X_train=df_train.drop(columns=label_columns,axis=1)
X_test=df_test

In [ ]:
features=list(X_train)
cat_features=['TypeOfSteel_A300','TypeOfSteel_A400']

scale_cols = features.copy()
for f in cat_features:
    scale_cols.remove(f)

# Preprocessing

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train[scale_cols] = scaler.fit_transform(X_train[scale_cols])
X_test[scale_cols] = scaler.transform(X_test[scale_cols])


# XGBoost CV

In [ ]:
class Model:
    def __init__(self, train, test):
        self.train             = train
        self.test              = test
        self.model_dict        = dict()
        self.test_predict_list = list()

    def fit(self, params):
        label_columns = ['Pastry','Z_Scratch','K_Scatch','Stains','Dirtiness','Bumps','Other_Faults']

        train_cols = [col for col in self.train.columns.to_list() if col not in label_columns]
        scores = list()

        for i in range(5):
            mskf = MultilabelStratifiedKFold(n_splits=10, shuffle=True)
            oof_valid_preds = np.zeros((self.train[train_cols].shape[0], len(label_columns)))

            for fold, (train_idx, valid_idx) in enumerate(mskf.split(self.train[train_cols], self.train[label_columns])):
                X_train, y_train = self.train[train_cols].iloc[train_idx], self.train[label_columns].iloc[train_idx]
                X_valid, y_valid = self.train[train_cols].iloc[valid_idx], self.train[label_columns].iloc[valid_idx]

                model = XGBClassifier(**params)
                model.fit(X_train, y_train, eval_set = [(X_valid, y_valid)], early_stopping_rounds = 250, verbose = False)

                valid_preds = model.predict_proba(X_valid)
                oof_valid_preds[valid_idx] = valid_preds

                test_predict = model.predict_proba(self.test[train_cols])
                self.test_predict_list.append(test_predict)

                score = roc_auc_score(y_valid, valid_preds, multi_class="ovr")
                self.model_dict[f'fold_{fold}'] = model

            oof_score = roc_auc_score(self.train[label_columns], oof_valid_preds, multi_class="ovr")
            print(f"The OOF auc score for iteration {i+1} is {oof_score}")
            scores.append(oof_score)
        return scores, self.test_predict_list, oof_valid_preds

# Tunned Parameters

In [ ]:
params_1 = {
    "gamma": 0.44,
    "verbosity": 0,
    "max_depth": 5,
    "subsample": 0.7,
    "reg_alpha": 0.54,
    "random_state": 18,
    "n_estimators": 1800,
    "min_child_weight": 4,
    "reg_lambda": 1.8e-06,
    "learning_rate": 0.006,
    "colsample_bytree": 0.38,
    "booster": "gbtree",
    "device_type": "cuda",
    "tree_method": "gpu_hist",
    "grow_policy": "depthwise",
}
params_2 = {
    "gamma": 0.24997920132991797,
    "verbosity": 0,
    "max_depth": 5,
    "subsample": 0.8841265541346639,
    "reg_lambda": 2.9660886967874625,
    "n_estimators": 703,
    "min_child_weight": 5,
    "reg_alpha": 0.00011509254946941848,
    "learning_rate": 0.023358116742747285,
    "colsample_bytree": 0.362499715714305,
    "booster": "gbtree",
    "device_type": "cuda",
    "tree_method": "gpu_hist",
    "grow_policy": "depthwise",
}

params_3 = {
    "verbosity": 0,
    "max_depth": 5,
    "n_estimators": 494,
    "random_state": 18,
    "min_child_weight": 3,
    "subsample": 0.8504122771965839,
    "reg_alpha": 1.5855453969671037e-06,
    "reg_lambda": 1.4155529076600075,
    "learning_rate": 0.02767540293640535,
    "colsample_bytree": 0.46589178614541227,
    "device": "cuda",
    "tree_method": "hist",
}

params_4 = {
    "max_depth": 7,
    "min_child_weight": 1,
    "n_estimators": 1235,
    "gamma": 0.6499918347241912,
    "subsample": 0.9116532305497375,
    "reg_lambda": 1.7005084366184795,
    "reg_alpha": 0.0059679946773570774,
    "learning_rate": 0.008352405007099802,
    "colsample_bytree": 0.49334879814671045,
    "device": "cuda",
    "tree_method": "hist"
}

params_5 = {
    "gamma": 0.44,
    "verbosity": 0,
    "max_depth": 5,
    "subsample": 0.7,
    "reg_alpha": 0.54,
    "random_state": 18,
    "n_estimators": 1800,
    "min_child_weight": 4,
    "reg_lambda": 1.8e-06,
    "learning_rate": 0.006,
    "colsample_bytree": 0.38,
    "booster": "gbtree",
    "tree_method": "gpu_hist",
    "grow_policy": "depthwise",
}

In [ ]:
X_train_ = pd.concat([X_train, y_train], axis=1,ignore_index=True)
X_train_.columns=features+label_columns

In [ ]:
model = Model(X_train_,X_test)
scores,preds_1,oof_valid_preds_1 = model.fit(params_1)
print(f'The average roc-auc score is {np.mean(scores)}')
score1=np.mean(scores)

# model = Model(X_train_,X_test)
# scores,preds_2 = model.fit(params_2)
# print(f'The average roc-auc score is {np.mean(scores)}')
# score2=np.mean(scores)

# model = Model(X_train_,X_test)
# scores,preds_3 = model.fit(params_3)
# print(f'The average roc-auc score is {np.mean(scores)}')
# score3=np.mean(scores)

# model = Model(X_train_,X_test)
# scores,preds_4 = model.fit(params_4)
# print(f'The average roc-auc score is {np.mean(scores)}')
# score4=np.mean(scores)

# model = Model(X_train_,X_test)
# scores,preds_5 = model.fit(params_5)
# print(f'The average roc-auc score is {np.mean(scores)}')
# score5=np.mean(scores)


# Feature Engineering

In [ ]:
feature_engineering = True
if feature_engineering:
    X_train_ = X_train_.drop(
        ['Y_Maximum','Y_Minimum',
         'SigmoidOfAreas','Edges_X_Index',
         'Sum_of_Luminosity','X_Perimeter'
        ],axis=1)

model = Model(X_train_,X_test)
scores,preds_6,oof_valid_preds_6 = model.fit(params_1)
print(f'The average roc-auc score is {np.mean(scores)}')
score1=np.mean(scores)

# Hypertuning Parameters

In [ ]:
!pip install optuna
!pip install --upgrade optuna

import optuna
!pip install cma
from functools import partial
from sklearn.metrics import log_loss
from optuna.samplers import CmaEsSampler

class OptunaWeights:
    def __init__(self, random_state, n_trials=5000):
        self.study = None
        self.weights = None
        self.random_state = random_state
        self.n_trials = n_trials

    def _objective(self, trial, y_true, y_preds):
        # Define the weights for the predictions from each model
        weights = [trial.suggest_float(f"weight{n}", 0, 1) for n in range(len(y_preds))]

        # Calculate the weighted prediction
        weighted_pred = np.average(np.array(y_preds), axis=0, weights=weights)

        auc_score = roc_auc_score(y_true, weighted_pred, multi_class="ovr")
        log_loss_score=log_loss(y_true, weighted_pred)
        return auc_score #/log_loss_score

    def fit(self, y_true, y_preds):
        optuna.logging.set_verbosity(optuna.logging.ERROR)
        sampler = optuna.samplers.CmaEsSampler(seed=self.random_state)
        pruner = optuna.pruners.HyperbandPruner()
        self.study = optuna.create_study(sampler=sampler, pruner=pruner, study_name="OptunaWeights", direction='maximize')
        objective_partial = partial(self._objective, y_true=y_true, y_preds=y_preds)
        self.study.optimize(objective_partial, n_trials=self.n_trials)
        self.weights = [self.study.best_params[f"weight{n}"] for n in range(len(y_preds))]


label_columns = ['Pastry','Z_Scratch','K_Scatch','Stains','Dirtiness','Bumps','Other_Faults']
ow = OptunaWeights(42)
ow.fit(X_train_[label_columns], y_preds = [
                                    oof_valid_preds_1, oof_valid_preds_6
                                ])
ow.weights

In [ ]:
import numpy as np
# change all weights to make sum equality to 0.3
lst_optuna_weights = np.asarray(ow.weights)
sum_lst_optuna_weights = np.sum(lst_optuna_weights)
lst_optuna_weights = lst_optuna_weights / (sum_lst_optuna_weights * (100/17))
lst_optuna_weights

# Submission

In [ ]:
df_int_result = pd.read_csv('/kaggle/input/steel-plate-fault-submission/submission_intermediate.csv')

In [ ]:
predictions_1 = np.mean(preds_1,axis=0)
# predictions_2 = np.mean(preds_2,axis=0)
# predictions_3 = np.mean(preds_3,axis=0)
# predictions_4 = np.mean(preds_4,axis=0)
# predictions_5 = np.mean(preds_5,axis=0)
predictions_6 = np.mean(preds_6,axis=0)



#predictions = w[0] * df_int_result[label_columns] + w[1] * predictions_1 + w[2] * predictions_2  + w[3] * predictions_3 + w[4] * predictions_4
w = [0.83]
predictions = (
    w[0] * df_int_result[label_columns] +
    (lst_optuna_weights[0]) * predictions_1 +
    (lst_optuna_weights[1]) * predictions_6
)

sub = pd.DataFrame(predictions, columns=label_columns)
sub['id'] = df_int_result['id']
sub.to_csv('submission.csv',index=False)
sub